In [21]:
import cv2
from matplotlib import pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
from PIL import Image
from PIL import ImageFilter
from matplotlib import animation
from PIL import ImageEnhance

from tkinter import filedialog
import tkinter as tk

import pickle

from os import sys

import matplotlib.pyplot as plt
import argparse

from statistics import mean
import os

%matplotlib inline

# for loading array 
from scipy.interpolate import interp1d
from scipy.io import loadmat
from scipy.signal import filtfilt, butter
from scipy import signal as sig
import pandas as pd
import numpy as np
import mat4py


def smooth(x,window_len=11,window='hanning'):
    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal
        
    example:

    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)
    
    see also: 
    
    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter
 
    TODO: the window parameter could be the window itself if an array instead of a string
    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
    """

    if x.ndim != 1:
        raise Exception("smooth only accepts 1 dimension arrays.")

    if x.size < window_len:
        raise Exception("Input vector needs to be bigger than window size.")


    if window_len<3:
        return x


    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise Exception("Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")


    s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    #print(len(s))
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')

    y=np.convolve(w/w.sum(),s,mode='valid')
    return y


def calculateNewSize(width,height,WishForNEwWidth):
    ratio = WishForNEwWidth/width
    Nheight = int(height * ratio)
    Nwidth = int(width * ratio)    
    return Nwidth , Nheight

def FindReferenceSystem(video):
    
    HandleBEHAV = cv2.VideoCapture(video_path, 0)
    length = int(HandleBEHAV.get(cv2.CAP_PROP_FRAME_COUNT))
   

    _ , IMG1 = HandleBEHAV.read()

    linemask= IMG1[140:141, : , 0]
    #print(np.shape(linemask))

    linemask = linemask.flatten()

    smoothmask = smooth(linemask)
    #print(np.shape(smoothmask))
    diffmask = np.diff(smoothmask,n=1)


    X = np.arange(0,np.size(diffmask),1)
    Xinterp = np.arange(0,np.size(diffmask),0.1)

    #print(np.shape(X))
    #print(Xinterp)

    f2 = interp1d(X, abs(diffmask), kind='cubic', fill_value="extrapolate")
    Peaks , values = sig.find_peaks(f2(Xinterp), height = 5.5)

    if np.size(Peaks) == 2 : 
        #ListObj1.append(Peaks[0])
        #ListObj2.append(Peaks[1])

        #print('2 peaks found at frame : {}, {}'.format(framne_nb, Peaks))

        Peaks1 = (((Peaks [0])*10**-1)  - 18)
        Peaks2 = (((Peaks [1])*10**-1)  + 12)
        return [Peaks1,Peaks2]
 
    
def FindReferenceSystem2(video):
    
    
    HandleBEHAV = cv2.VideoCapture(video_path, 0)
    length = int(HandleBEHAV.get(cv2.CAP_PROP_FRAME_COUNT))
    
    _ , IMG1 = HandleBEHAV.read()

    linemask2= IMG1[600:601, : , 0]

    linemask2 = np.invert(linemask2)
    linemask2 = linemask2.flatten()

    smoothmask = smooth(linemask2)

    b, a = butter(8, 0.075)
    smoothfiltered = filtfilt(b, a, smoothmask)

    diffmask = np.diff(smoothfiltered,n=1)

    PeaksNO2 , values = sig.find_peaks(diffmask, height = 1)

    if np.size(PeaksNO2) == 1 : 
        Peaks3 = ((PeaksNO2 [0])  - 10)
        Peaks4 = ((PeaksNO2 [0])  + 10)
        return [Peaks3,Peaks4]


In [20]:

video_path = r"C:\Users\nkhefif\Desktop\Extraits_BHV_test_hits\Mouse24_2020-02-10T14.42.09.avi"

HandleBEHAV = cv2.VideoCapture(video_path, 0)
#HandleVSD = cv2.VideoCapture(r'C:\Users\Nicolas\Desktop\Mouse17_Correl_BVH_VSD_test\Session4_trial4\S04-T04_2.mov')

length = int(HandleBEHAV.get(cv2.CAP_PROP_FRAME_COUNT))


#1 objet
ListObj1 = []
ListObj2 = []

#2eme objet
ListObj3 = []
ListObj4 = []



#1er objet
Reference_peaks = FindReferenceSystem(video_path)
#2eme objet
Reference_peaks2 = FindReferenceSystem2(video_path)

    
print(Reference_peaks[0])
print(Reference_peaks[1])
print(Reference_peaks2[0])


#####2eme objet#####

for framne_nb in range(length-1) :


    _ , IMG1 = HandleBEHAV.read()

    linemask = IMG1[600:601, int(Reference_peaks2[0]):int(Reference_peaks2[1])]

    lineIMG = linemask.copy()

    linemask = linemask.flatten()

    smoothmask = smooth(linemask)

    diffmask = np.diff(smoothmask,n=1)

    X = np.arange(0,np.size(diffmask),1)
    Xinterp = np.arange(0,np.size(diffmask),0.1)

    f2 = interp1d(X, abs(diffmask), kind='cubic', fill_value="extrapolate")
    DPeaks , values = sig.find_peaks(f2(Xinterp), height = 1)

    if np.size(DPeaks) == 2:
         
        ListObj3.append(DPeaks[0])
        ListObj4.append(DPeaks[1])

            
    else :
       
        ListObj3.append(np.nan)
        ListObj4.append(np.nan)




print(DPeaks)
print(ListObj3)
print(ListObj4)

#on cree les parametres tresholding pour les var d intensite des pics de la liste ListObj1 en +/- car fct abs sur l affichage du subplot
#donc l'analyse se fait sur les deux valeurs

Listpeaks2 = []
Listframe2 = []    

SD = np.nanstd(ListObj)
MeanPeaks = np.nanmean(ListObj1)



##### On sort les valeurs des peaks différents à un treshold déterminé par app à la moyenne +/-SD, de manière à pouvoir être fait sur chaque vidéo ###
for framne_nb in range(length-1) :

    if ListObj1[framne_nb] >= (MeanPeaks + (4 * SD)) :
        Listframe.append(framne_nb)
        Listpeaks.append(ListObj1[framne_nb])

    elif ListObj1[framne_nb] <= (MeanPeaks - (4 * SD)) :
        Listframe.append(framne_nb)
        Listpeaks.append(ListObj1[framne_nb])






####1er objets#####""

for framne_nb in range(length-1) :


    _ , IMG1 = HandleBEHAV.read()

    linemask2 = IMG1[140:141, int(Reference_peaks[0]):int(Reference_peaks[1])]
    
    ##############

    linemask3 = linemask2

    lineIMG = linemask3.copy()

    linemask3 = linemask3.flatten()

    smoothmask = smooth(linemask3)

    diffmask = np.diff(smoothmask,n=1)

    X = np.arange(0,np.size(diffmask),1)
    Xinterp = np.arange(0,np.size(diffmask),0.1)

    f2 = interp1d(X, abs(diffmask), kind='cubic', fill_value="extrapolate")
    Peaks , values = sig.find_peaks(f2(Xinterp), height = 5.5)

    if np.size(DPeaks) == 2:
         
        ListObj1.append(Peaks[0])
        ListObj2.append(Peaks[1])
            #print('2 peaks found at frame : {}, {}'.format(framne_nb, Peaks))
            
    else :
       
            ListObj1.append(np.nan)
            ListObj2.append(np.nan)




print(DPeaks)
print(ListObj1[0])
print(ListObj1)
print(ListObj2)

#on cree les parametres tresholding pour les var d intensite des pics de la liste ListObj1 en +/- car fct abs sur l affichage du subplot
#donc l'analyse se fait sur les deux valeurs

Listpeaks = []
Listframe = []    

SD = np.nanstd(ListObj1)
MeanPeaks = np.nanmean(ListObj1)


print('La standard deviation est de :', (SD))
print('La moyenne des peaks est de :', (MeanPeaks))

##### On sort les valeurs des peaks différents à un treshold déterminé par app à la moyenne +/-SD, de manière à pouvoir être fait sur chaque vidéo ###
for framne_nb in range(length-1) :

    if ListObj1[framne_nb] >= (MeanPeaks + (4 * SD)) :
        Listframe.append(framne_nb)
        Listpeaks.append(ListObj1[framne_nb])

    elif ListObj1[framne_nb] <= (MeanPeaks - (4 * SD)) :
        Listframe.append(framne_nb)
        Listpeaks.append(ListObj1[framne_nb])

print('La liste des frames inclues dans l\'interval (mean +/- 4 * SD) est :', (Listframe))  
print('La liste des peaks correspondant est :', (Listpeaks))


if len(Listframe) == 0: 
    pass
else:
    outfile = os.path.join(os.path.dirname(video_path), os.path.basename(video_path) [:-4] + '_Hits.pckl')

    with open(outfile,'wb') as pickleHandle:
        whatever = pickle.dump(Listframe, pickleHandle)


    
    
    
    
    
fig = plt.figure(figsize = (25,35))

plt.subplot(221), plt.plot(abs(diffmask))

#plt.subplot(222), plt.plot(abs(smooth(np.asarray(ListObj1)-MeanPeaks)))

plt.subplot(222), plt.plot(np.asarray(ListObj1))
#plt.xticks(range(0, 540, 20))
#plt.yticks(range(200, 500, 30))
plt.grid(axis="x")

plt.subplot(223), plt.imshow(lineIMG, 'gray')

plt.subplot(224), plt.plot(Listframe, Listpeaks, 'o')
plt.xticks(range(0, 500, 20))
#plt.yticks(range(200, 700, 10))
plt.grid(axis="x")


    #plt.subplot(224), plt.plot(Xinterp,f2(Xinterp))

            #linemask = np.swapaxes(linemask,0, 1)
            #plt.ylim(255, 0)
            #print(Xinterp)





@interact

def show_image(time=widgets.IntSlider(min=0,max=length,step=1,value=1), \
Brightness=widgets.FloatSlider(min=0,max=4,step=0.015,value=1), \
binarization=widgets.FloatSlider(min=-1,max=2,step=0.01,value=-1), \
Denoise = False, VSD=False, substract= False, EnhanceEdge=False, EnhanceEdgeMore=False ):


    HandleBEHAV.set(cv2.CAP_PROP_POS_FRAMES, time)
    _ , BEHi = HandleBEHAV.read()


    #HandleVSD.set(cv2.CAP_PROP_POS_FRAMES, time)
    #_ , VSDi = HandleVSD.read() 
    #VSDi = cv2.cvtColor(VSDi, cv2.COLOR_BGR2RGB)

    output = BEHi
    fig = plt.figure(figsize = (20,30))
    plt.imshow(output, interpolation='none')
    #cv2.line(HandleBEHAV, 240, 540, 'r', 1)



327.1
434.70000000000005
1238
[]
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, n

NameError: name 'ListObj' is not defined